# Lab 06: Spatial smoothing, denoising with ICA, and temporal filtering

The goal of this lab is to practice common pre-preprocessing steps that all involve some form of filtering out signal that is deemed to be "noise" based on signal processing concepts. We will look at three common steps that follow motion correction and slice-timing: 
* Spatial smoothing
* ICA-based denoising
* Temporal filtering